

A simple nice solution with tensorflow. (0.7446)
------------------------------------------------



In [1]:
# Import the libraries
import pandas as pd
import re
import tensorflow as tf
from tensorflow.contrib import learn
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Library for tensorflow logging
import logging
logging.getLogger().setLevel(logging.INFO)
df = pd.read_csv("../input/train.csv")

In [3]:
# Selecting features
features = ["bone_length","rotting_flesh","hair_length","color","has_soul"]
X = df[features]
y = df["type"]

In [4]:
# Encoding type (Ghost,Ghouls,Goblin) and color
from sklearn.preprocessing import LabelEncoder as LE
letype = LE()
y = letype.fit_transform(y)
lecolor = LE()
X["color"] = lecolor.fit_transform(X["color"])

In [5]:
# splitting function used for cross validation
from sklearn.cross_validation import train_test_split
# current test size = 0 to permit the usage of whole training data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.0, random_state=9)

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
# define a network with a single hidden RELU layer of 15 hidden units 
tf_clf_dnn = learn.DNNClassifier(hidden_units=[15], n_classes=3)
tf_clf_dnn.fit(X_train, y_train,max_steps=5500)
from sklearn.metrics import accuracy_score as as_
# print(as_(y_test,tf_clf_dnn.predict(X_test)))
print(as_(y_train,tf_clf_dnn.predict(X_train)))

Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float32)
INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 1.24366
INFO:tensorflow:Step 101: loss = 0.861342
INFO:tensorflow:Step 201: loss = 0.707243
INFO:tensorflow:Saving checkpoints for 300 into /tmp/tmpnm79rdll/model.ckpt.
INFO:tensorflow:Step 301: loss = 0.634195
INFO:tensorflow:Step 401: loss = 0.596028
INFO:tensorflow:Step 501: loss = 0.571143
INFO:tensorflow:Saving checkpoints for 600 into /tmp/tmpnm79rdll/model.ckpt.
INFO:tensorflow:Step 601: loss = 0.55352
INFO:tensorflow:Step 701: loss = 0.541385
INFO:tensorflow:Step 801: loss = 0.532201
INFO:tensorflow:Saving checkpoints for 900

0.749326145553


In [7]:
# Reading csv into test_df
test_df = pd.read_csv("../input/test.csv")
X_test = test_df[features]

# Reading ID
id_ = test_df["id"]

# Encoding color
X_test["color"] = lecolor.transform(X_test["color"])
 
# Prediction and Decoding into labels
pred = tf_clf_dnn.predict(X_test)
pred = letype.inverse_transform(pred)
output = pd.DataFrame({"id": id_,"type":pred})
output.to_csv('ghostPred.csv',index=False)

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float32)
INFO:tensorflow:Loading model from checkpoint: /tmp/tmpnm79rdll/model.ckpt-5500-?????-of-00001.
